# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x165578950>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x162605a00>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x165578950>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x162605a00>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x165578950>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 146 events
"Project Created" occurred at 2024-05-13T11:38:24.571284+02:00
"Project Created" occurred at 2024-05-14T09:54:33.141388+02:00
"Project Created" occurred at 2024-05-14T09:54:33.149371+02:00
"Project Created" occurred at 2024-05-14T09:54:33.152154+02:00
"Project Created" occurred at 2024-05-14T09:54:33.154889+02:00
"Project Created" occurred at 2024-05-14T09:54:33.157795+02:00
"Project Created" occurred at 2024-05-14T09:54:33.161230+02:00
"Project Created" occurred at 2024-05-14T09:54:33.165007+02:00
"Project Created" occurred at 2024-05-14T09:54:33.167719+02:00
"Project Created" occurred at 2024-05-14T09:54:33.170376+02:00
"Project Created" occurred at 2024-05-14T09:54:33.172675+02:00
"Project Created" occurred at 2024-05-14T09:54:46.633757+02:00
"Project Created" occurred at 2024-05-14T10:01:07.276888+02:00
"Project Created" occurred at 2024-05-14T10:05:23.831958+02:00
"Project Created" occurred at 2024-05-15T08:51:33.765023+02:00
"Project Created" occurred at 2024-05-

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 146 events
Reset projection
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 30 project summaries
Project 494949ef-dd5c-4eb9-8155-70ceff675942: Rio Primero
Project eb19084e-fb17-4440-9139-f8d24db2e267: Project 0
Project a2a44d01-0a99-4674-87ed-e02494f0ef54: Project 1
Project 6f0288b8-9493-4f1e-8d4e-76efb41b0c3e: Project 2
Project 9f190cb6-6ece-43c1-8a36-48a43c398976: Project 3
Project dfe2a535-4530-45d4-afda-c0300411fea4: Project 4
Project 7e0fc7c6-cdae-4b78-a681-a5922ea8205f: Project 5
Project a392705a-357d-4840-b695-31188e347383: Project 6
Project 6f5cd25d-aeee-4ec1-92c5-4424eb0b8161: Project 7
Project 101183ee-9f7b-4372-ae52-55ee21fea8b8: Project 8
Project b3f777e6-8a0e-46b7-9119-8469ee71452c: Project 9
Project 9c392b58-67c8-4958-a284-d64cdf70c21b: Project 1
Project c8016bd0-e219-40bc-80fb-3c2501d7cbcd: Rio Primero 123
Project aba1deaa-3baf-4626-8793-d8acf666edf9: Rio Primero 123
Project 50c86de5-94ee-4287-a18a-8d5909c62884: Project 1
Project 0be79a0c-2be7-426d-be57-b4503c71f799: Project 1
Project 84dade68-0939-4b0a-ab44-2ebff0c8b8a4: Project 1
Project